In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import gmaps
import gmaps.datasets
import gmaps
import gmaps.datasets
from IPython.display import display
import time
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets


In [ ]:
#Creates lists of the center of the field for every session, can be used to center the map to the correct field

#Path do HIR.csv file
HIR_path = ''

# Load the sprint data from the CSV file
df = pd.read_csv(HIR_path, delimiter=',', usecols=["Session_Id", "Lat_start", "Lat_end", "Lon_start", "Lon_end"])

grouped = df.groupby('Session_Id').agg({
    'Lat_start': 'mean',
    'Lon_start': 'mean',
    'Lat_end': 'mean',
    'Lon_end': 'mean'
})

grouped['center_lat'] = (grouped['Lat_start'] + grouped['Lat_end']) / 2
grouped['center_lon'] = (grouped['Lon_start'] + grouped['Lon_end']) / 2

print(grouped[['center_lat', 'center_lon']])


In [ ]:
#Draw one HIR on map

#Google maps API key
google_api_key = ''
gmaps.configure(api_key=google_api_key)

df = pd.read_csv('Calculated values/HIR.csv', delimiter=',', usecols=["Lat_start", "Lat_end", "Lon_start", "Lon_end"])

fig = gmaps.figure(center=(63.44518212047379, 10.451604763140454), zoom_level=18, map_type='SATELLITE')


for i in range(0, 5):
    Lat_start = df.Lat_start[i]
    Lon_start = df.Lon_start[i]
    Lat_end = df.Lat_end[i]
    Lon_end = df.Lon_end[i]


    # Features to draw on the map
    gmt_meridian = gmaps.Line(
        start=(Lat_start, Lon_start),
        end=(Lat_end, Lon_end),
        stroke_weight=5,
        stroke_color="red"
    )


    drawing = gmaps.drawing_layer(features=[gmt_meridian])
    fig.add_layer(drawing)
display(fig)


In [ ]:
import pandas as pd
import gmaps

#Google maps API key
google_api_key = ''
gmaps.configure(api_key=google_api_key)

# Read the CSV file (assuming it has a "session_id" column)
HIR_path = ''
df = pd.read_csv(HIR_path, delimiter=',', usecols=["Session_Id", "Lat_start", "Lat_end", "Lon_start", "Lon_end"])

# Set your desired session_id here
desired_session_id = "2020-06-01-TeamA-bcc03f81-2733-45d3-abf1-f7a709c63e68"

# Filter the DataFrame to only include rows with the desired session_id
filtered_df = df[df['Session_Id'] == desired_session_id]

# Calculate the center of the map using the filtered data
center_lat = (filtered_df['Lat_start'].mean() + filtered_df['Lat_end'].mean()) / 2
center_lon = (filtered_df['Lon_start'].mean() + filtered_df['Lon_end'].mean()) / 2

fig = gmaps.figure(center=(center_lat, center_lon), zoom_level=18, map_type='SATELLITE')

for _, row in filtered_df.iterrows():
    Lat_start = row['Lat_start']
    Lon_start = row['Lon_start']
    Lat_end = row['Lat_end']
    Lon_end = row['Lon_end']

    # Features to draw on the map
    gmt_meridian = gmaps.Line(
        start=(Lat_start, Lon_start),
        end=(Lat_end, Lon_end),
        stroke_weight=5,
        stroke_color="red"
    )

    drawing = gmaps.drawing_layer(features=[gmt_meridian])
    fig.add_layer(drawing)

display(fig)


In [ ]:
#Define time range for plotting timelaps

import datetime

#Path to player data parquet.
player_path = ''
table = pq.read_table(player_path, columns=['time', 'lat', 'lon'])
df = table.to_pandas()
df = df.iloc[::100, :]

dates = []
for i in range (0, len(df.index) -1, 10):
    dates.append(df.time[i])

#options = [(i.strftime("%H:%M:%S"), i) for i in dates]
range_selector = widgets.SelectionRangeSlider(
    options=dates,
    index=(0,len(dates)-1),
    description='Time',
    disabled=False,
    
    layout=Layout(width='50%', height='80px')
)

display(range_selector)




In [ ]:
#Timelaps one player one session


start = df.time[df.time == range_selector.value[0]].index.tolist()[-1]
end = df.time[df.time == range_selector.value[1]].index.tolist()[0]

fig = gmaps.figure(center=(63.44518212047379, 10.451604763140454), zoom_level=18, map_type='SATELLITE')

drawing = gmaps.drawing_layer()
fig.add_layer(drawing)
display(fig)
time.sleep(2)

prev_lat = 0
Prev_lon = 0
for i in range (start, end):
    drawing.features = [    
        gmaps.Marker((df.lat[i], df.lon[i])) 
    ]
    time.sleep(1)

In [ ]:

#Heatmap

fig = gmaps.figure(map_type='SATELLITE')

locations= []

table_length = len(df.index)
for i in range(0, len(df.index) - 1):
    locations.append((df.lat[i], df.lon[i]))

heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

